In [ ]:
import tensorflow as tf
print(tf.__version__)
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from stable_baselines3 import PPO , DDPG, SAC,TD3 #,RecurrentPPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import torch

# Check if a GPU is available
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Get the number of available GPUs
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

# Print the name of each available GPU
for i in range(num_gpus):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# Load the pre-trained models
crop_parameters_estimator = load_model('/media/ai-ws2/8f2a1bf6-3409-412b-abae-522c4615e68f/ImanHindi/AGHC/AutonomousGreenHouseChallenge/Code/Training/wcp_LSTM_model_fs_model.h5')
resource_consumption_estimator = load_model('/media/ai-ws2/8f2a1bf6-3409-412b-abae-522c4615e68f/ImanHindi/AGHC/AutonomousGreenHouseChallenge/Code/Training/rc_LSTM_model_fs_model.h5')
gh_climate_estimator = load_model('/media/ai-ws2/8f2a1bf6-3409-412b-abae-522c4615e68f/ImanHindi/AGHC/AutonomousGreenHouseChallenge/Code/Training/ghc_mlp_model_fs_model.h5')

rewards = []
CP_important_feature=[ 'Tair', 'pH_drain_PC', 'Cum_irr', 't_heat_vip', 'water_sup', 'Tot_PAR', 'water_sup_intervals_vip_min', 'PipeGrow', 'EC_drain_PC',
                        'BlackScr', 'co2_dos', 'Tot_PAR_Lamps', 'scr_enrg_vip', 'Rhair', 'HumDef', 'days']

GH_C_important_feature=['PARout',  'Tout',  'Iglob',  'RadSum',  'scr_enrg_vip',  't_heat_vip',  'int_white_vip',  'scr_blck_vip',  'pH_drain_PC',  'co2_vip',
                        't_ventlee_vip',  'days']

RC_important_feature=[     'Cum_irr',  'BlackScr',  'water_sup_intervals_vip_min',  'EC_drain_PC',  'pH_drain_PC',  'CO2air',  'water_sup',  'HumDef',
                            'Rhair',   'Tot_PAR' ]

actions_sp=['co2_vip', 'int_white_vip', 'pH_drain_PC','scr_blck_vip', 'scr_enrg_vip',  't_heat_vip',
            't_ventlee_vip', 'water_sup','water_sup_intervals_vip_min',  'days'] #10 Actions 

important_ghc=['BlackScr','CO2air','Cum_irr','EC_drain_PC','PipeGrow','HumDef','Rhair','Tair','Tot_PAR','Tot_PAR_Lamps']

weather_sp=[ 'PARout',   'Tout', 'Iglob',  'RadSum']

GH_C_Out_columns=['AssimLight','BlackScr','CO2air','Cum_irr','EC_drain_PC','EnScr','HumDef','PipeGrow','PipeLow','Rhair','Tair','Tot_PAR',
                  'Tot_PAR_Lamps','VentLee','Ventwind' ,'assim_vip','co2_dos' ]
class GreenhouseEnv(gym.Env):
    def __init__(self, crop_parameters_estimator, resource_consumption_estimator, gh_climate_estimator,weather_data):
        super(GreenhouseEnv, self).__init__()
        self.crop_parameters_estimator = crop_parameters_estimator
        self.resource_consumption_estimator = resource_consumption_estimator
        self.gh_climate_estimator=gh_climate_estimator
        # Action space: shape (2016, 34) - 2016 time steps for 34 control setpoints
        self.action_space = spaces.Box(low=0,high=1,shape=(2016 * 9,),dtype=np.float64) #Flattened shape: 2016*34
    

        self.observation_space = spaces.Dict({
            'weather': spaces.Box(low=0, high=1, shape=(2016, 10), dtype=np.float64),
            'crop_params': spaces.Box(low=0, high=1, shape=(1, 3), dtype=np.float64),
            'resource_consumption': spaces.Box(low=0, high=7, shape=(1, 5), dtype=np.float64),
            'gh_climate': spaces.Box(low=-10, high=10, shape=(2016, 10), dtype=np.float64)
        })
        
        # Initial state
        self.weather_data = weather_data
        self.current_step = 0
        self.steps = 0
        self.max_steps = 23
        self.days=np.array([(i // 288 ) / 166  for i in range(2016*(self.max_steps))]).reshape(2016*(self.max_steps), 1)

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        self.steps = 0
        self.days=np.array([(i // 288 ) / 166  for i in range(2016*(self.max_steps))]).reshape(2016*(self.max_steps), 1)
        
        initial_crop_params = np.zeros(3).reshape(1, 3)+np.random.uniform(0, .1, size=(1,3))
        initial_resource_consumption = np.zeros(5).reshape(1, 5)+np.random.uniform(0, .1, size=(1,5))
        initial_daily_resource_consumption = np.zeros(shape=(7,1,5)).reshape(7,1,5)+np.random.uniform(0, .1, size=(7,1,5))
        initial_gh_climate=np.zeros(shape=(1,2016,10)).reshape(2016,10)+np.random.uniform(0, .1, size=(2016,10))
        # Return the initial state
        self.state = {
            'weather': np.array(self.weather_data[self.steps * 2016:(self.steps + 1) * 2016]),
            'crop_params': initial_crop_params,
            'resource_consumption': initial_resource_consumption,
            'gh_climate': initial_gh_climate
        }
        print(self.state['weather'].shape)
        self.daily_res_cons=initial_daily_resource_consumption

        return self.state, {}

    def step(self, action):
        action = action.reshape((2016, 9))
        day = np.array(self.days[self.steps * 2016:(self.steps + 1) * 2016]).reshape(2016, 1) #np.array([(i // 288 + 1) / 166 * (self.steps + 1) for i in range(2016)]).reshape(2016, 1)
        self.steps += 1
        print('day',day)
        control_setpoints = np.concatenate([action.reshape(2016, 9), day], axis=1)
                
        control_setpoints = pd.DataFrame(control_setpoints, columns=actions_sp)
        print(type(self.state['weather']))
        print(type(control_setpoints))
        weather=pd.DataFrame(self.state['weather'],columns=w_columns)
        #predict gh_climate:
        w_sp_data = pd.concat([weather, control_setpoints], axis=1)
        GH_C_Estimator_Input=np.array(w_sp_data[GH_C_important_feature])

        print('w_sp_data',w_sp_data.shape)
        ghclimate=self.gh_climate_estimator.predict(GH_C_Estimator_Input).reshape(2016,17)
        gh_climate=pd.DataFrame(ghclimate,columns=GH_C_Out_columns)
        
        # Predict crop parameters using the crop LSTM model
        CP_Estimator_Input = np.array(pd.concat([gh_climate, control_setpoints], axis=1)[CP_important_feature]).reshape(1,2016,16)
        weekly_crop_params = self.crop_parameters_estimator.predict(CP_Estimator_Input).reshape(1,3)

        RC_Estimator_Input = np.array(pd.concat([gh_climate, control_setpoints], axis=1)[RC_important_feature]).reshape(7, 288, 10)
        
        daily_resource_consumption = self.resource_consumption_estimator.predict(RC_Estimator_Input)
        print(RC_Estimator_Input.shape) #(7, 288, 11)

        print(daily_resource_consumption.shape) #(7, 288, 5)
        self.daily_res_cons=daily_resource_consumption
        weekly_resource_consumption = self.resource_consumption_estimator.predict(RC_Estimator_Input).sum(axis=0).reshape(1, 5)
        print(weekly_resource_consumption.shape)
        for i in self.resource_consumption_estimator.predict(RC_Estimator_Input)[:]:
            high_rc = np.any(i>=1)
            if high_rc:
                break
        # Update state
        self.state = {
            'weather': self.weather_data[self.steps * 2016:(self.steps + 1) * 2016],
            'crop_params': weekly_crop_params,
            'resource_consumption': weekly_resource_consumption,
            'gh_climate': np.array(gh_climate[important_ghc]).reshape(2016,10)
        }
        print(weekly_crop_params[0].shape)
        print(weekly_resource_consumption[0].shape)
        print(weekly_resource_consumption)
        # Calculate reward
        reward = self.calculate_reward(weekly_crop_params[0], weekly_resource_consumption[0],high_rc,action)
        done = bool((self.steps >= self.max_steps) or np.any(weekly_crop_params[0]<0.2 ) or np.any(weekly_resource_consumption[0]>7) or np.all(weekly_crop_params[0]<0.5) ) #or high_rc)
        
        return self.state, reward, done, done, {}
    def calculate_reward(self, crop_params, resource_consumption,high_rc,current_actions):
        punishment=0
        big_reward=0
        alpha, beta, delta,gamma = 1, 1, 0.02,0.002 # Adjusted delta and gamma values
        w1, w2, w3 = 1, 1, 1
        p1, p2, p3, p4 = 1, 1, 1, 1
        max_stem_elong, max_stem_thick, max_cum_trusses = 1.0, 1.0, 1.0
        max_heat, max_co2, max_electricity, max_irrigation = 7.0, 7.0, 7.0, 7.0

        crop_reward = (w1 * (crop_params[0] / max_stem_elong) +
                       w2 * (crop_params[1] / max_stem_thick) +
                       w3 * (crop_params[2] / max_cum_trusses))
        resource_penalty = (p1 * (resource_consumption[0] / max_heat) +
                            p2 * (resource_consumption[1] / max_co2) +
                            p3 * ((resource_consumption[2] + resource_consumption[3]) / max_electricity) +
                            p4 * (resource_consumption[4] / max_irrigation))

        if np.any(resource_consumption>7):
            print('resource_consumption>7',np.any(resource_consumption>7))
            punishment-=1
        if np.any(crop_params<.5):
            print('crop_params<.5',np.any(crop_params<.5))
            punishment-=0.8
        
        if np.all(crop_params<.5):
            print('all crop_params<.5',np.all(crop_params<.5))
            punishment-=1
        if np.any(crop_params>=.7):
            print('crop_params>=.7',np.any(crop_params>=.7))
            big_reward+=.5
        if np.any(crop_params>=.8):
            print('crop_params>=.8',np.any(crop_params>=.8))
            big_reward+=.8
        if np.all(crop_params>=.5):
            print('all crop_params>=.5',np.all(crop_params>=.5))
            big_reward+=.8
        if np.all(crop_params>=.7):
            print('all crop_params>=.7',np.all(crop_params>=.7))
            big_reward+=.8
        if np.all(crop_params>=.8):
            print('all crop_params>=.8',np.all(crop_params>=.8))
            big_reward+=1
        efficiency_factor = crop_reward / (1 + resource_penalty)
        print(efficiency_factor,'efficiency_factor')
        #Stability penalty
        s = 0.01
        max_delta_action = 16.0  # Assuming actions are normalized between 0 and 1
        stability=0
        for idx in range(len(current_actions)-1):
            stability+=np.sum(np.abs(current_actions[idx]-current_actions[idx+1]))/ max_delta_action
        stability_penalty = s * stability

        reward = alpha * crop_reward - beta * resource_penalty +punishment+big_reward + delta * efficiency_factor - gamma * stability_penalty
        print(reward)
        return reward
    
# Assuming weather_data is a preprocessed time series array for the environment
weather_data = pd.read_csv('/media/ai-ws2/8f2a1bf6-3409-412b-abae-522c4615e68f/ImanHindi/AGHC/AutonomousGreenHouseChallenge/Code/Training/weather_fill_missing_values.csv',index_col='%time')#pd.read_csv('D:\\Iman\\AGHC\\CherryTomato\\Data\\Weather\\Weather.csv',index_col='%time') #  # Example placeholder data #np.random.rand(48384, 10)  # Placeholder data
w_columns=weather_data.columns
print(weather_data.shape)
scaler=MinMaxScaler()
weather_data=scaler.fit_transform(weather_data)

print(weather_data.shape)
# Random generation of weather data
weather_data = np.random.rand(48384, 10)

# Initialize the environment
env = GreenhouseEnv(crop_parameters_estimator, resource_consumption_estimator,gh_climate_estimator, weather_data)
check_env(env)

import os
# Define a folder for saving logs_fs and models
log_dir = "./TD3_logs_fs/"
os.makedirs(log_dir, exist_ok=True)

# Wrap the environment in a monitor to track performance
env = Monitor(env, log_dir)

In [ ]:
# Get the number of actions in the environment (for noise generation)
n_actions = env.action_space.shape[-1]
# Define action noise for exploration (Ornstein-Uhlenbeck noise)
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=0.2 * np.ones(n_actions), theta=0.15)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print(device)
# Define the action noise for exploration (NormalActionNoise)
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

# Optimized TD3 hyperparameters
TD3_model = TD3(
    "MultiInputPolicy",  # Use the default MLP policy
    env, 
    verbose=1,  # Print training progress
    learning_rate=1e-4,  # Learning rate for the actor and critic networks
    buffer_size=100_000,  # Replay buffer size
    batch_size=256,  # Batch size for training
    tau=0.005,  # Soft update coefficient for target networks
    gamma=0.99,  # Discount factor for future rewards
    train_freq=(1, "episode"),  # Train after every episode
    gradient_steps=100,  # Number of gradient steps to perform after each episode
    learning_starts=1000,  # Start learning after 10,000 timesteps
    policy_delay=2,  # Policy update delay
    target_policy_noise=0.2,  # Target smoothing noise for the next state
    target_noise_clip=0.5,  # Clip noise
    action_noise=action_noise,  # Exploration noise
    policy_kwargs=dict(net_arch=[256, 256,128]),  # Network architecture for the actor and critic networks
    tensorboard_log=log_dir,  # Optional: Tensorboard log directory
    device=device
)


In [ ]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

# Create evaluation callback to monitor agent performance every 100k steps
eval_callback = EvalCallback(env, best_model_save_path='./TD3_logs_fs/best_model/',
                             log_path='./TD3_logs_fs/results/', eval_freq=10_000, deterministic=True, render=False)

# Save models every 100k steps
checkpoint_callback = CheckpointCallback(save_freq=10_000, save_path='./TD3_logs_fs/checkpoints/', name_prefix='TD3_greenhouse')


# Train the model
TD3_model.learn(total_timesteps=100_000, callback=[eval_callback, checkpoint_callback])

# Save the trained model
TD3_model.save("TD3_greenhouse_final_model")

# Evaluate the model
mean_reward, std_reward = evaluate_policy(TD3_model, env, n_eval_episodes=10, deterministic=True)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

In [ ]:

from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

# Create evaluation callback to monitor agent performance every 100k steps
eval_callback = EvalCallback(env, best_model_save_path='./TD3_logs_fs/best_model/',
                             log_path='./TD3_logs_fs/results/', eval_freq=10000, deterministic=True, render=False)

# Save models every 100k steps
checkpoint_callback = CheckpointCallback(save_freq=10000, save_path='./TD3_logs_fs/checkpoints/', name_prefix='TD3_greenhouse')

import os
from stable_baselines3 import TD3

# Define the path to the checkpoint directory
checkpoint_dir = './TD3_logs_fs/checkpoints/'

# latest_checkpoint = checkpoints[latest_checkpoint_index]
latest_checkpoint_path = os.path.join(checkpoint_dir, 'TD3_greenhouse_50000_steps.zip')

# Load the model from the latest checkpoint
print(f"Loading model from {latest_checkpoint_path}")
TD3_model = TD3.load(latest_checkpoint_path, env=env)

# Continue training
TD3_model.learn(total_timesteps=80_000, callback=[eval_callback, checkpoint_callback])  # Adjust timesteps as needed

# Optionally save the model again after continuing training
TD3_model.save("TD3_greenhouse_final_model_after_resume")

# Evaluate the model
mean_reward, std_reward = evaluate_policy(TD3_model, env, n_eval_episodes=10, deterministic=True)
print(f"Mean reward: {mean_reward} +/- {std_reward}")




In [ ]:
import os
from stable_baselines3 import PPO,DDPG,TD3

# Define the path to the best model directory
best_model_dir = r'C:\Users\Iman.Hindi\Desktop\AACSGH_Paper\models\weights'

# Check if the best model file exists
best_model_filename = 'TD3_greenhouse_final_model.zip'  # Change this to your actual best model filename if different
best_model_path = os.path.join(best_model_dir, best_model_filename)

# Load the best model
if os.path.exists(best_model_path):
    print(f"Loading the best model from {best_model_path}")
    best_model = TD3.load(best_model_path, env=env)
else:
    raise FileNotFoundError(f"No best model found at {best_model_path}")

# Optionally, evaluate the best model
mean_reward, std_reward = evaluate_policy(best_model, env, n_eval_episodes=10, deterministic=True)
print(f"Mean reward of the best model: {mean_reward} +/- {std_reward}")


In [ ]:

from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback


import os
from stable_baselines3 import TD3

# Define the path to the checkpoint directory
checkpoint_dir = 'C:\\Users\\Iman.Hindi\\Desktop\\AACSGH_Paper\\results\\best_training_logs\\TD3_logs_fs\\checkpoints\\'

latest_checkpoint_path = os.path.join(checkpoint_dir, 'TD3_greenhouse_50000_steps.zip')

# Load the model from the latest checkpoint
best_model = TD3.load(latest_checkpoint_path)

# Continue training
#TD3_model.learn(total_timesteps=80_000, callback=[eval_callback, checkpoint_callback])  # Adjust timesteps as needed

# Optionally save the model again after continuing training
best_model.save("TD3_greenhouse_final_model_after_resume")

# Evaluate the model
mean_reward, std_reward = evaluate_policy(best_model,  n_eval_episodes=10, deterministic=True)
print(f"Mean reward: {mean_reward} +/- {std_reward}")




In [ ]:
import pandas as pd 
orgin_path = "/media/ai-ws2/8f2a1bf6-3409-412b-abae-522c4615e68f/ImanHindi/AGHC/AutonomousGreenHouseChallenge/Code/Training/"
filenames = ['AICU', 'Automatoes',
             'Digilog', 'IUACAAS', 
             'Reference', 'TheAutomators'
             ]

T1_Action=pd.read_csv(orgin_path+f'{filenames[0]}Actions.csv',index_col='%time')
T2_Action=pd.read_csv(orgin_path+f'{filenames[1]}Actions.csv',index_col='%time')
T3_Action=pd.read_csv(orgin_path+f'{filenames[2]}Actions.csv',index_col='%time')
T4_Action=pd.read_csv(orgin_path+f'{filenames[3]}Actions.csv',index_col='%time')
T5_Action=pd.read_csv(orgin_path+f'{filenames[4]}Actions.csv',index_col='%time')
T6_Action=pd.read_csv(orgin_path+f'{filenames[5]}Actions.csv',index_col='%time')


T1_Results=pd.read_csv(orgin_path+f'{filenames[0]}Results.csv',index_col='%time')[:18]
T2_Results=pd.read_csv(orgin_path+f'{filenames[1]}Results.csv',index_col='%time')[:18]
T3_Results=pd.read_csv(orgin_path+f'{filenames[2]}Results.csv',index_col='%time')[:18]
T4_Results=pd.read_csv(orgin_path+f'{filenames[3]}Results.csv',index_col='%time')[:18]
T5_Results=pd.read_csv(orgin_path+f'{filenames[4]}Results.csv',index_col='%time')[:18]
T6_Results=pd.read_csv(orgin_path+f'{filenames[5]}Results.csv',index_col='%time')[:18]

T1_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[0]}resource_consumption.csv',index_col='%time')
T2_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[1]}resource_consumption.csv',index_col='%time')
T3_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[2]}resource_consumption.csv',index_col='%time')
T4_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[3]}resource_consumption.csv',index_col='%time')
T5_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[4]}resource_consumption.csv',index_col='%time')
T6_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[5]}resource_consumption.csv',index_col='%time')

actions_sp=['co2_vip', 'int_white_vip', 'pH_drain_PC','scr_blck_vip', 'scr_enrg_vip',  't_heat_vip',
            't_ventlee_vip', 'water_sup','water_sup_intervals_vip_min',  'days'] #10 Actions 
resources=['Heat_cons','ElecHigh','ElecLow', 'CO2_cons','Irr']
crop_param=['Stem_elong' ,'Stem_thick','Cum_trusses']

episode_rewards = []
#model_actions=np.empty(action.shape)
model_results=[]
model_r_consumption=[]
env = GreenhouseEnv(crop_parameters_estimator, resource_consumption_estimator,gh_climate_estimator, weather_data)
check_env(env)
state = env.reset(seed=42)[0]
done = False
episode_reward = 0
i=0
model_action=[]
while not done:
        action = best_model.predict(state,deterministic=True)[0]
        if i==0:
          model_actions=action 
          print('model_actions',model_actions)  
        else: 
          model_actions=np.concatenate([model_actions,action],axis=0)
        print(model_actions)
        print(model_actions.shape)
        i+=1
        #model_actions=np.concatenate([np.array(model_actions),action],axis=0)
        state, reward, done, _, _ = env.step(action)
        model_results.append(state['crop_params'])
        model_r_consumption.append(state['resource_consumption'])
        model_action.append(action)
        episode_reward += reward
episode_rewards.append(episode_reward.astype('float32'))
print(f'Episode Reward: {episode_reward.astype("float32")}')
sp=actions_sp
sp_noday=['co2_vip', 'int_white_vip', 'pH_drain_PC','scr_blck_vip', 'scr_enrg_vip',  't_heat_vip',
            't_ventlee_vip', 'water_sup','water_sup_intervals_vip_min'] #10 Actions 
model_actions.shape
model_results=pd.DataFrame(np.array(model_results).reshape(23,3),columns=crop_param)[:18]
model_r_consumption=pd.DataFrame(np.array(model_r_consumption).reshape(23,5),columns=resources)
model_actions=pd.DataFrame(np.array(model_action).reshape(23*2016,9),columns=sp_noday)
model_results.set_index(T1_Results.index,inplace=True)
model_r_consumption.set_index(T1_resource_Consumption[:-1].index,inplace=True)

model_actions=model_actions[:len(T5_Action.index)].set_index(T5_Action.index[:len(model_actions.index)])

model_actions.index
teams=['AICU', 'Automatoes',
             'Digilog', 'IUACAAS', 
             'Reference', 'TheAutomators','Ours']
teams_Results={ teams[0]: T1_Results,
                 teams[1]: T2_Results,
                 teams[2]: T3_Results,
                 teams[3]: T4_Results,
                 teams[4]: T5_Results,
                 teams[5]: T6_Results,
                 teams[6]:  model_results,
                    
                    }
teams_Actions={ teams[0]: T1_Action[actions_sp],
                teams[1]: T2_Action[actions_sp],
                teams[2]: T3_Action[actions_sp],
                teams[3]: T4_Action[actions_sp],
                teams[4]: T5_Action[actions_sp],
                teams[5]: T6_Action[actions_sp],
                teams[6]:  model_actions[sp_noday],
                    }
teams_rc={      teams[0]: T1_resource_Consumption,
                teams[1]: T2_resource_Consumption,
                teams[2]: T3_resource_Consumption,
                teams[3]: T4_resource_Consumption,
                teams[4]: T5_resource_Consumption,
                teams[5]: T6_resource_Consumption,
                teams[6]:  model_r_consumption,
    
}
#plot Teams results
print(teams_Results[teams[0]].columns)
#plot Teams actions
import matplotlib.pyplot as plt
for result in teams_Results[teams[0]].columns:
    for i in range(len(teams)):
        teams_Results[teams[i]][result].plot(ylabel=result,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{result}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{result}_time_series_plot ')

    plt.show()
#plot Teams actions
# import matplotlib.pyplot as plt
# for action in teams_Actions[teams[5]].columns:
#     for i in range(len(teams)):
#         teams_Actions[teams[i]]["2019-12-16 00:00:00" : "2019-12-17 00:00:00"][action].plot(ylabel=action,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
#     plt.title(f'green_house_{action}_time_series_plot for all Teams ')
#     plt.legend()
#     #save_fig(f'green_house_{action}_time_series_plot ')

#     plt.show()
#plot Teams rc

#plot Teams actions
import matplotlib.pyplot as plt
for resource in teams_rc[teams[0]].columns[:-1]:
    for i in range(len(teams)):
        teams_rc[teams[i]][resource].plot(ylabel=resource,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{resource}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{resource}_time_series_plot ')

    plt.show()
teams_episode_rewards = {}
for team in teams:
    print(f'Evaluate {team} Actions....')
    
    teams_model_actions=np.empty((2016, 10))
    Our_model_actions=np.empty((2016, 9))
    
    model_results=[]
    model_r_consumption=[]
    state = env.reset(seed=42)[0]
    done = False
    episode_reward = []
    i=0
    sp=actions_sp
    while not done:
            if team=='Ours':
                action = best_model.predict(state,deterministic=True)[0]
                Our_model_actions=np.concatenate([np.array(Our_model_actions),np.array(action).reshape(2016,9)],axis=0)
                state, reward, done, _, _ = env.step(np.array(action))
            else:
                action = teams_Actions[team].iloc[2016*i:2016*(i+1)][sp]
                teams_model_actions=np.concatenate([np.array(teams_model_actions),np.array(action)],axis=0)
                print(action.shape)
                state, reward, done, _, _ = env.step(np.array(action.drop('days',axis=1)).reshape(2016*9,))
            i+=1
            
            
            model_results.append(state['crop_params'][0])
            model_r_consumption.append(state['resource_consumption'])
            episode_reward.append(reward.astype('float32'))
    print(episode_reward)
    teams_episode_rewards[team]=np.array(episode_reward)
    print(f'{team} Episode Reward: {np.array(teams_episode_rewards[team]).sum().astype("float32")}')
print(model_results)



In [ ]:
import pandas as pd 
orgin_path = "/media/ai-ws2/8f2a1bf6-3409-412b-abae-522c4615e68f/ImanHindi/AGHC/AutonomousGreenHouseChallenge/Code/Training/"
filenames = ['AICU', 'Automatoes',
             'Digilog', 'IUACAAS', 
             'Reference', 'TheAutomators'
             ]

T1_Action=pd.read_csv(orgin_path+f'{filenames[0]}Actions.csv',index_col='%time')
T2_Action=pd.read_csv(orgin_path+f'{filenames[1]}Actions.csv',index_col='%time')
T3_Action=pd.read_csv(orgin_path+f'{filenames[2]}Actions.csv',index_col='%time')
T4_Action=pd.read_csv(orgin_path+f'{filenames[3]}Actions.csv',index_col='%time')
T5_Action=pd.read_csv(orgin_path+f'{filenames[4]}Actions.csv',index_col='%time')
T6_Action=pd.read_csv(orgin_path+f'{filenames[5]}Actions.csv',index_col='%time')


T1_Results=pd.read_csv(orgin_path+f'{filenames[0]}Results.csv',index_col='%time')[:18]
T2_Results=pd.read_csv(orgin_path+f'{filenames[1]}Results.csv',index_col='%time')[:18]
T3_Results=pd.read_csv(orgin_path+f'{filenames[2]}Results.csv',index_col='%time')[:18]
T4_Results=pd.read_csv(orgin_path+f'{filenames[3]}Results.csv',index_col='%time')[:18]
T5_Results=pd.read_csv(orgin_path+f'{filenames[4]}Results.csv',index_col='%time')[:18]
T6_Results=pd.read_csv(orgin_path+f'{filenames[5]}Results.csv',index_col='%time')[:18]

T1_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[0]}resource_consumption.csv',index_col='%time')
T2_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[1]}resource_consumption.csv',index_col='%time')
T3_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[2]}resource_consumption.csv',index_col='%time')
T4_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[3]}resource_consumption.csv',index_col='%time')
T5_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[4]}resource_consumption.csv',index_col='%time')
T6_resource_Consumption=pd.read_csv(orgin_path+f'{filenames[5]}resource_consumption.csv',index_col='%time')

actions_sp=['co2_vip', 'int_white_vip', 'pH_drain_PC','scr_blck_vip', 'scr_enrg_vip',  't_heat_vip',
            't_ventlee_vip', 'water_sup','water_sup_intervals_vip_min',  'days'] #10 Actions 
resources=['Heat_cons','ElecHigh','ElecLow', 'CO2_cons','Irr']
crop_param=['Stem_elong' ,'Stem_thick','Cum_trusses']

episode_rewards = []
#model_actions=np.empty(action.shape)
model_results=[]
model_r_consumption=[]
env = GreenhouseEnv(crop_parameters_estimator, resource_consumption_estimator,gh_climate_estimator, weather_data)
check_env(env)
state = env.reset(seed=42)[0]
done = False
episode_reward = 0
i=0

while not done:
        action = best_model.predict(state,deterministic=True)[0]
        print(action.shape)
        if i==0:
          model_actions=action.reshape(2016,9)
        else: 
          model_actions=np.concatenate([model_actions,action.reshape(2016,9)],axis=0)
        print(model_actions.shape)
        i+=1
        #model_actions=np.concatenate([np.array(model_actions),action],axis=0)
        state, reward, done, _, _ = env.step(action)
        model_results.append(state['crop_params'])
        model_r_consumption.append(state['resource_consumption'])
        episode_reward += reward
episode_rewards.append(episode_reward.astype('float32'))
print(f'Episode Reward: {episode_reward.astype("float32")}')
sp=actions_sp
sp_noday=['co2_vip', 'int_white_vip', 'pH_drain_PC','scr_blck_vip', 'scr_enrg_vip',  't_heat_vip',
            't_ventlee_vip', 'water_sup','water_sup_intervals_vip_min'] #10 Actions 
model_actions.shape
model_results=pd.DataFrame(np.array(model_results).reshape(23,3),columns=crop_param)[:18]
model_r_consumption=pd.DataFrame(np.array(model_r_consumption).reshape(23,5),columns=resources)
model_actions=pd.DataFrame(np.array(model_actions),columns=sp_noday)
model_results.set_index(T1_Results.index,inplace=True)
model_r_consumption.set_index(T1_resource_Consumption[:-1].index,inplace=True)

model_actions=model_actions[:len(T5_Action.index)].set_index(T5_Action.index[:len(model_actions.index)])

model_actions.index
teams=['AICU', 'Automatoes',
             'Digilog', 'IUACAAS', 
             'Reference', 'TheAutomators','Ours']
teams_Results={ teams[0]: T1_Results,
                 teams[1]: T2_Results,
                 teams[2]: T3_Results,
                 teams[3]: T4_Results,
                 teams[4]: T5_Results,
                 teams[5]: T6_Results,
                 teams[6]:  model_results,
                    
                    }
teams_Actions={ teams[0]: T1_Action[actions_sp],
                teams[1]: T2_Action[actions_sp],
                teams[2]: T3_Action[actions_sp],
                teams[3]: T4_Action[actions_sp],
                teams[4]: T5_Action[actions_sp],
                teams[5]: T6_Action[actions_sp],
                teams[6]:  model_actions[sp_noday],
                    }
teams_rc={      teams[0]: T1_resource_Consumption,
                teams[1]: T2_resource_Consumption,
                teams[2]: T3_resource_Consumption,
                teams[3]: T4_resource_Consumption,
                teams[4]: T5_resource_Consumption,
                teams[5]: T6_resource_Consumption,
                teams[6]:  model_r_consumption,
    
}
#plot Teams results
print(teams_Results[teams[0]].columns)
#plot Teams actions
import matplotlib.pyplot as plt
for result in teams_Results[teams[0]].columns:
    for i in range(len(teams)):
        teams_Results[teams[i]][result].plot(ylabel=result,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{result}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{result}_time_series_plot ')

    plt.show()


#plot Teams actions
import matplotlib.pyplot as plt
for resource in teams_rc[teams[0]].columns[:-1]:
    for i in range(len(teams)):
        teams_rc[teams[i]][resource].plot(ylabel=resource,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{resource}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{resource}_time_series_plot ')

    plt.show()
teams_episode_rewards = {}
all_teams_results={}
all_teams_r_consumption={}
all_teams_episode_rewards={}
for team in teams:
    print(f'Evaluate {team} Actions....')
    
    teams_model_actions=np.empty((2016, 9))
    Our_model_actions=np.empty((2016, 9))
    
    model_results=[]
    
    model_r_consumption=[]
    state = env.reset(seed=42)[0]
    done = False
    episode_reward = []
    i=0
    sp=actions_sp
    while not done:
            if team=='Ours':
                action = best_model.predict(state,deterministic=True)[0]
                Our_model_actions=np.concatenate([np.array(Our_model_actions),np.array(action).reshape(2016,9)],axis=0)
                state, reward, done, _, _ = env.step(np.array(action))
                
            else:
                action = teams_Actions[team].iloc[2016*i:2016*(i+1)][sp_noday]
                teams_model_actions=np.concatenate([np.array(teams_model_actions),np.array(action)],axis=0)
                print(action.shape)
                state, reward, done, _, _ = env.step(np.array(action))
            i+=1
            
            
            model_results.append(state['crop_params'])
            model_r_consumption.append(state['resource_consumption'])
            episode_reward.append(reward.astype('float32'))
           
    model_results=pd.DataFrame(np.array(model_results).reshape(23,3),columns=crop_param)[:18]
    model_r_consumption=pd.DataFrame(np.array(model_r_consumption).reshape(23,5),columns=resources)
    model_actions=pd.DataFrame(np.array(model_actions),columns=sp_noday)
    model_results.set_index(T1_Results.index,inplace=True)
    model_r_consumption.set_index(T1_resource_Consumption[:-1].index,inplace=True)
    all_teams_results[team]=model_results
    all_teams_r_consumption[team]=model_r_consumption
    print(episode_reward)
    teams_episode_rewards[team]=np.array(episode_reward)
    print(f'{team} Episode Reward: {np.array(teams_episode_rewards[team]).sum().astype("float32")}')
    all_teams_episode_rewards[team]=np.array(teams_episode_rewards[team]).sum().astype("float32")



In [ ]:
#plot Teams results
print(all_teams_results[teams[0]].columns)
line=['*','o','>','^','x','v','.']

#plot Teams actions
import matplotlib.pyplot as plt
for result in all_teams_results[teams[0]].columns:
    for i in range(len(teams)):
        all_teams_results[teams[i]][result].plot(ylabel=result,grid=True, marker=line[i], figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{result}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{result}_time_series_plot ')

    plt.show()

#plot Teams actions
line=['*','o','>','^','x','v','.']

import matplotlib.pyplot as plt
for resource in all_teams_r_consumption[teams[0]].columns[:]:
    for i in range(len(teams)):
        #all_teams_r_consumption[teams[i]]['Irr']=all_teams_r_consumption[teams[i]]['Irr']*(.0001*x)+all_teams_r_consumption[teams[i]]['Irr']
        all_teams_r_consumption[teams[i]][resource].plot(ylabel=resource,grid=True, marker=line[i], figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{resource}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{resource}_time_series_plot ')

    plt.show()


In [ ]:
import pandas as pd
data_df=pd.read_csv('./irrigation_time_series.csv')
markers = ['*', 'o', '>', '^', 'x', 'v', '.']
# Replotting with the correct column names
plt.figure(figsize=(12, 6))
for i, team in enumerate(data_df.columns[1:]):  # Skip 'Time' column
    plt.plot(data_df['Timestamp'], data_df[team], marker=markers[i], label=team)

plt.title("Greenhouse Irrigation Time Series Plot for All Teams")
plt.xlabel("Time")
plt.ylabel("Irrigation (Irr)")
plt.legend(title="Teams", loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
all_teams_results

In [ ]:
all_teams_r_consumption

In [ ]:
import matplotlib.pyplot as plt
for result in teams_Results[teams[0]].columns:
    for i in range(len(teams)):
        teams_Results[teams[i]][result].plot(ylabel=result,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{result}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{result}_time_series_plot ')

    plt.show()

#plot Teams actions
import matplotlib.pyplot as plt
for resource in teams_rc[teams[0]].columns[:-1]:
    for i in range(len(teams)):
        teams_rc[teams[i]][resource].plot(ylabel=resource,grid=True, marker=".", figsize=(18, 6),legend=True,label=f'{teams[i]}')
    plt.title(f'green_house_{resource}_time_series_plot for all Teams ')
    plt.legend()
    #save_fig(f'green_house_{resource}_time_series_plot ')

    plt.show()


In [ ]:
print(teams_episode_rewards['Ours'])


In [ ]:

rewards=[]
for team_reward in teams_episode_rewards:
    print(team_reward)
    print(len(teams_episode_rewards[team_reward]))
    print(teams_episode_rewards[team_reward].sum())
    rewards.append(teams_episode_rewards[team_reward].sum())
teams_episode_rewards_df=pd.DataFrame(np.array(rewards).reshape(1,7),columns=teams)
print(teams_episode_rewards_df)
final_result=pd.DataFrame(teams_episode_rewards_df.sum(axis=0).sort_values(ascending=False))
final_result
import matplotlib.pyplot as plt

plt.bar(final_result.index, final_result[0])
plt.xticks(rotation=90)  # Rotate the x-axis labels by 90 degrees
plt.show()


In [ ]:
plt.bar(final_result.index, final_result[0])
plt.xticks(rotation=90)  # Rotate the x-axis labels by 90 degrees
plt.show()

In [ ]:
import matplotlib.pyplot as plt
mean_rc={}

for i in range(len(teams)):
        mean_rc[teams[i]]=all_teams_r_consumption[teams[i]].sum(axis=0)[['CO2_cons','ElecHigh','ElecLow','Heat_cons','Irr']].round(2)

mean_rc_df=pd.DataFrame.from_dict(mean_rc)
mean_rc_df.to_csv('sum_rc_df_data_TD3.csv', index=True)  # Set index=True to include the index column
mean_rc

In [ ]:
import matplotlib.pyplot as plt
mean_cp={}

for i in range(len(teams)):
        mean_cp[teams[i]]=all_teams_results[teams[i]].mean(axis=0).round(2)

mean_cp
mean_cp_df=pd.DataFrame.from_dict(mean_cp)
mean_cp_df.to_csv('mean_cp_df_data_TD3.csv', index=True)  # Set index=True to include the index column
mean_cp
